In [ ]:
%matplotlib nbagg
import IPython.display
import os
# ----
import json
import tempfile
import subprocess
import matplotlib.pyplot as plt
# ----
import numpy as np
import pandas as pd
import statsmodels.api as sm


Code for importing JSON logs into pandas

In [ ]:
def make_df(rows, t0=None) :
    pd1       = pd.DataFrame.from_records(rows, columns=['at','msg'])
    pd1['at'] = pd.to_datetime(pd1['at'])
    if t0 is not None:
        pd1['dt'] = (pd1['at'] - t0).astype('timedelta64[ms]')
    pd2       = pd.DataFrame([l['data'] for l in rows])
    return pd.concat([pd1, pd2], axis=1)    
    
def load_log(fname) :
    "Load JSON log data as list of dictionaries"
    with open(fname) as f :
        return [ json.loads(s) for s in f.readlines()]

def decode_log(rows) :
    "Load JSON logs for mempool"
    t0 = pd.to_datetime(rows[0]['at'])
    rMempool = make_df([l for l in rows if l['ns'] == ['TM','mempool']    ], t0)
    rCmtD    = make_df([l for l in rows if l['msg'] == 'Decision to commit'], t0)
    rCmtA    = make_df([l for l in rows if l['msg'] == 'Actual commit'     ], t0)
    rGsp     = make_df([l for l in rows if l['msg'] == 'Gossip stats'], t0)
    return {'after'  : rMempool[rMempool['msg'] == 'Mempool after filtering'].reset_index(),
            'before' : rMempool[rMempool['msg'] == 'Mempool before filtering'].reset_index(),
            'commit' : pd.DataFrame({'t1':rCmtD['at'],
                                     't2':rCmtA['at'],
                                     'dt1' : rCmtD['dt'],
                                     'dt2' : rCmtA['dt'],
                                     'Ntx':rCmtA['Ntx']
                                    },
                                    index=rCmtD['H']).dropna(),
            'gossip' : rGsp
           }

def read_one_trace(rows, i) :
    "Generate trace for single trace output"
    msgC  = 'Entering new height ----------------'
    # Commit trace
    rowsC = [l for l in rows if l['msg'] == msgC]
    dfC         = make_df(rowsC)
    dfC['node'] = i
    # Steps trace
    rowsS       = [l for l in rows if l['msg'].startswith('Entering') ]
#                                  and l['msg'] != 'Entering new height ----------------']
    dfS         = make_df(rowsS)
    dfS['node'] = 1000 + i
    dfS         = dfS.fillna(value={'reason':''})
    # Merge traces
    dfC = pd.DataFrame(data={'at':dfC['at'], 'node':dfC['node'], 'msg':dfC['H'] % 2})
    dfS = pd.DataFrame(data={'at':dfS['at'], 'node':dfS['node'], 'msg':dfS['msg']+' '+dfS['reason']})
    return pd.concat([dfC,dfS]).sort_values(by='at').reset_index()

def read_trace(logs) :
    "Generate traces for all logs"
    return pd.concat([read_one_trace(l,i) for i,l in enumerate(logs)]).reset_index()

def splot(trace, w=1900) :
    with tempfile.NamedTemporaryFile(mode='w', suffix='.trace') as f :
        for i in [0,1,2,3] :
            print( "%s <NODE%i XXX" % (trace['at'][0], i), file=f.file)
            print( "%s <NODE%i XXX" % (trace['at'][0], 1000+i), file=f.file)
        for _,q in trace.iterrows() :
            print( "%s >NODE%i %s" % (q['at'], q['node'], q['msg'] ), file=f.file)
        with tempfile.NamedTemporaryFile(suffix='.png') as out :
            subprocess.run(["splot", "-w", str(w), "-h", str(4*2*30), "-bh 20", "-if", f.name, "-o", out.name],
                           check=True)
            return IPython.display.Image(out.name, unconfined=True)

Constant data

In [ ]:
logs = ['../ser2/logs/node-1',
        '../ser2/logs/node-2',
        '../ser2/logs/node-3',
        '../ser2/logs/node-4',
       ]
logData = [load_log(n)   for n in logs]
dfs     = [decode_log(n) for n in logData]

# Blocks plots

In [ ]:
def fun():
    plt.figure()
    plt.grid()
    plt.title("Block size")
    plt.xlabel("Height")
    plt.ylabel("N of transactions")
    tot = np.sum(dfs[0]['commit']['Ntx'])
    avg = np.average(dfs[0]['commit']['Ntx'])
    for df in dfs :
        plt.plot(df['commit'].index, df['commit']['Ntx'])
    plt.axhline(y=avg, color='k')
    print( "Total transactions commited: ", tot )
    print( "Transactions per block:      ", avg )
fun()

In [ ]:
def fun():
    # Do linear fit of commit times which are averaged over all nodes
    n  = np.min([df['commit']['dt2'].shape[0] for df in dfs])
    hs = np.asarray( dfs[0]['commit'].index )[0:n]
    ts = np.average( [df['commit']['dt2'][0:n] for df in dfs], axis=0 )
    r = sm.OLS(ts, sm.add_constant(hs), missing='drop').fit()
    #
    fig = plt.figure()
    plt.grid()
    plt.title('Commit time')
    plt.xlabel('Time (ms)')
    plt.ylabel('Height')
    plt.plot(hs*r.params[1] + r.params[0], hs, color='grey', linewidth=0.5)
    for df in dfs :
        plt.plot(df['commit']['dt2'] , df['commit'].index, '+')
    print("Time for commit of singe block %.3f s" % float(r.params[1] / 1000))
    return fig

fun().savefig('plot.png')
None

# Mempool plots

In [ ]:
def fun():
    plt.figure()
    plt.grid()
    plt.title("Number of transaction added to mempool")
    for df in dfs :
        plt.plot(df['after']['at'], df['after']['added'], '+')
        plt.plot(df['after']['at'], df['after']['discarded'], 'x')
#        plt.plot(df['after']['at'], df['after']['filtered'], '.')
fun()

In [ ]:
def fun():
    plt.figure()
    plt.grid()
    plt.title("Mempool size")
    for df in dfs :
        plt.plot(df['after']['size'], '+-.')
fun()

In [ ]:
plt.figure()
plt.grid()
plt.title("Number of transactions filtered after commit")
for df in dfs :
    plt.plot(df['after']['filtered'] - df['before']['filtered'], '+')

In [ ]:
def fun(key):
    for df in dfs :
        plt.figure()
        plt.grid()
        plt.title("Cumulative number of transactions")
        plt.plot(df['gossip']['at'], df['gossip']['Rx'+key], label='Rx' )
        plt.plot(df['gossip']['at'], df['gossip']['Tx'+key], label='Tx' )
        plt.legend()
fun('Tx')

In [ ]:
splot(read_trace(logData))

In [ ]:
dfs